In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gc
import sys

from pandas.io.json import json_normalize
from datetime import datetime
from sklearn import preprocessing

import os

AttributeError: module 'os' has no attribute 'getwd'

In [4]:
print(os.getcwd())

/Users/aziz.aldawood/Documents/google_analytics


In [5]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column].tolist())
        column_as_df.columns = [f'{column}.{subcolumn}' for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
%%time
train = load_df(os.getcwd()+'/train.csv')
test = load_df(os.getcwd()+'/test.csv')

In [ ]:
print('train date:', min(train['date']), 'to', max(train['date']))
print('test date:', min(test['date']), 'to', max(test['date']))

In [7]:
for c in train.columns.values:
    if c not in test.columns.values: print(c)

totals.transactionRevenue
trafficSource.campaignCode


In [8]:
# totals, the sub-column transactionRevenue contains the revenue information we are trying to predict
train_rev = train[~train['totals.transactionRevenue'].isnull()].copy()
print(len(train_rev))
train_rev.head()

11515


,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
752,Direct,20160902,6194193421514403509,6194193421514403509_1472843572,Not Socially Engaged,1472843572,1,1472843572,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,NaN,(none),NaN,(direct)
753,Organic Search,20160902,5327166854580374902,5327166854580374902_1472844906,Not Socially Engaged,1472844906,3,1472844906,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google
799,Referral,20160902,8885051388942907862,8885051388942907862_1472827393,Not Socially Engaged,1472827393,7,1472827393,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,NaN,referral,/,mall.googleplex.com
802,Referral,20160902,0185467632009737931,0185467632009737931_1472846398,Not Socially Engaged,1472846398,6,1472846398,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,NaN,referral,/,mall.googleplex.com
859,Referral,20160902,3244885836845029978,3244885836845029978_1472824817,Not Socially Engaged,1472824817,4,1472824817,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,NaN,referral,/,mall.googleplex.com


In [9]:
train['totals.transactionRevenue'].fillna(0, inplace=True)
train['totals.transactionRevenue'] = np.log1p(train['totals.transactionRevenue'].astype(float))
print(train['totals.transactionRevenue'].describe())

count    903653.000000
mean          0.227118
std           2.003710
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          23.864375
Name: totals.transactionRevenue, dtype: float64


In [27]:
all_data = train.append(test, sort=False).reset_index(drop=True)

In [11]:
print(all_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708337 entries, 0 to 1708336
Data columns (total 55 columns):
channelGrouping                                      object
date                                                 int64
fullVisitorId                                        object
sessionId                                            object
socialEngagementType                                 object
visitId                                              int64
visitNumber                                          int64
visitStartTime                                       int64
device.browser                                       object
device.browserSize                                   object
device.browserVersion                                object
device.deviceCategory                                object
device.flashVersion                                  object
device.isMobile                                      bool
device.language                                      

In [12]:
null_cnt = train.isnull().sum().sort_values()
print(null_cnt[null_cnt > 0])

totals.pageviews                                   100
totals.newVisits                                200593
totals.bounces                                  453023
trafficSource.keyword                           502929
trafficSource.referralPath                      572712
trafficSource.isTrueDirect                      629648
trafficSource.adwordsClickInfo.gclId            882092
trafficSource.adwordsClickInfo.adNetworkType    882193
trafficSource.adwordsClickInfo.isVideoAd        882193
trafficSource.adwordsClickInfo.page             882193
trafficSource.adwordsClickInfo.slot             882193
trafficSource.adContent                         892707
trafficSource.campaignCode                      903652
dtype: int64


  channelGrouping      date        fullVisitorId  \
0  Organic Search  20160902  1131660440785968503   
1  Organic Search  20160902   377306020877927890   
2  Organic Search  20160902  3895546263509774583   
3  Organic Search  20160902  4763447161404445595   
4  Organic Search  20160902    27294437909732085   

                        sessionId  socialEngagementType     visitId  \
0  1131660440785968503_1472830385  Not Socially Engaged  1472830385   
1   377306020877927890_1472880147  Not Socially Engaged  1472880147   
2  3895546263509774583_1472865386  Not Socially Engaged  1472865386   
3  4763447161404445595_1472881213  Not Socially Engaged  1472881213   
4    27294437909732085_1472822600  Not Socially Engaged  1472822600   

   visitNumber  visitStartTime device.browser             device.browserSize  \
0            1      1472830385         Chrome  not available in demo dataset   
1            1      1472880147        Firefox  not available in demo dataset   
2            1      

In [25]:
all_data['totals.pageviews'].fillna(1, inplace=True)
all_data['totals.newVisits'].fillna(0, inplace=True)
all_data['totals.bounces'].fillna(0, inplace=True)
all_data['totals.pageviews'] = all_data['totals.pageviews'].astype(int)
all_data['totals.newVisits'] = all_data['totals.newVisits'].astype(int)
all_data['totals.bounces'] = all_data['totals.bounces'].astype(int)

# fillna boolean feature
all_data['trafficSource.isTrueDirect'].fillna(False, inplace=True)

0    9
1    9
2    9
3    9
4    9
Name: _month, dtype: int64


In [28]:
format_str = '%Y%m%d'
all_data['formated_date'] = all_data['date'].apply(lambda x: datetime.strptime(str(x), format_str))
all_data['_month'] = all_data['formated_date'].apply(lambda x:x.month)
all_data['_quarterMonth'] = all_data['formated_date'].apply(lambda x:x.day//8)
all_data['_day'] = all_data['formated_date'].apply(lambda x:x.day)
all_data['_weekday'] = all_data['formated_date'].apply(lambda x:x.weekday())

all_data.drop(['date','formated_date'], axis=1, inplace=True)

In [29]:
print(all_data['channelGrouping'].value_counts())

Organic Search    738963
Social            354971
Direct            273134
Referral          211307
Display            51283
Paid Search        45627
Affiliates         32915
(Other)              137
Name: channelGrouping, dtype: int64


In [30]:
all_data['_visitStartHour'] = all_data['visitStartTime'].apply(
    lambda x: str(datetime.fromtimestamp(x).hour))

In [31]:
all_data['totals.hits'] = all_data['totals.hits'].astype(int)
all_data['_meanHitsPerDay'] = all_data.groupby(['_day'])['totals.hits'].transform('mean')
all_data['_meanHitsPerWeekday'] = all_data.groupby(['_weekday'])['totals.hits'].transform('mean')
all_data['_meanHitsPerMonth'] = all_data.groupby(['_month'])['totals.hits'].transform('mean')
all_data['_sumHitsPerDay'] = all_data.groupby(['_day'])['totals.hits'].transform('sum')
all_data['_sumHitsPerWeekday'] = all_data.groupby(['_weekday'])['totals.hits'].transform('sum')
all_data['_sumHitsPerMonth'] = all_data.groupby(['_month'])['totals.hits'].transform('sum')

In [33]:
all_data['_adContentGMC'] = (all_data['trafficSource.adContent'] == 'Google Merchandise Collection').astype(np.uint8)

In [34]:
all_data['_withCampaign'] = (all_data['trafficSource.campaign'] != '(not set)').astype(np.uint8)

In [35]:
all_data['_sourceGpmall'] = (all_data['trafficSource.source'] == 'mall.googleplex.com').astype(np.uint8)

In [36]:
all_data.head()

,channelGrouping,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,...,_visitStartHour,_meanHitsPerDay,_meanHitsPerWeekday,_meanHitsPerMonth,_sumHitsPerDay,_sumHitsPerWeekday,_sumHitsPerMonth,_adContentGMC,_withCampaign,_sourceGpmall
0,Organic Search,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,not available in demo dataset,...,11,4.492082,4.56607,4.67738,241966,1092277,740462,0,0,0
1,Organic Search,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,not available in demo dataset,...,1,4.492082,4.56607,4.67738,241966,1092277,740462,0,0,0
2,Organic Search,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,not available in demo dataset,...,21,4.492082,4.56607,4.67738,241966,1092277,740462,0,0,0
3,Organic Search,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,not available in demo dataset,...,1,4.492082,4.56607,4.67738,241966,1092277,740462,0,0,0
4,Organic Search,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,not available in demo dataset,...,9,4.492082,4.56607,4.67738,241966,1092277,740462,0,0,0


In [39]:
all_data[all_data['totals.transactionRevenue'].notnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 903653 entries, 0 to 903652
Data columns (total 68 columns):
channelGrouping                                      903653 non-null object
fullVisitorId                                        903653 non-null object
sessionId                                            903653 non-null object
socialEngagementType                                 903653 non-null object
visitId                                              903653 non-null int64
visitNumber                                          903653 non-null int64
visitStartTime                                       903653 non-null int64
device.browser                                       903653 non-null object
device.browserSize                                   903653 non-null object
device.browserVersion                                903653 non-null object
device.deviceCategory                                903653 non-null object
device.flashVersion                                  903

In [40]:
train_df = all_data[all_data['totals.transactionRevenue'].notnull()]

In [41]:
train_df.to_csv('train_df.csv')

In [2]:
train_df.head()

NameError: name 'train_df' is not defined